In [94]:
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

set_config(display='diagram')

In [95]:
df = pd.DataFrame(np.random.randint(0,1000,size=(10000, 4)), columns=list('ABCD'))
df.head()

,A,B,C,D
0,936,76,68,859
1,725,903,732,752
2,689,617,356,771
3,182,996,338,290
4,179,236,191,536


In [96]:
df1 = pd.DataFrame()
for idx1,col1 in enumerate(df.columns):
    for idx2,col2 in enumerate(df.drop(columns=df.columns[:idx1]).columns):
        df1[col1+'x'+col2] = df.loc[:,col1] * df.loc[:,col2]

df1.head()

,AxA,AxB,AxC,AxD,BxB,BxC,BxD,CxC,CxD,DxD
0,876096,71136,63648,804024,5776,5168,65284,4624,58412,737881
1,525625,654675,530700,545200,815409,660996,679056,535824,550464,565504
2,474721,425113,245284,531219,380689,219652,475707,126736,274476,594441
3,33124,181272,61516,52780,992016,336648,288840,114244,98020,84100
4,32041,42244,34189,95944,55696,45076,126496,36481,102376,287296


In [97]:
df2 = df.merge(df1, left_index=True, right_index=True)

df2.head()

,A,B,C,D,AxA,AxB,AxC,AxD,BxB,BxC,BxD,CxC,CxD,DxD
0,936,76,68,859,876096,71136,63648,804024,5776,5168,65284,4624,58412,737881
1,725,903,732,752,525625,654675,530700,545200,815409,660996,679056,535824,550464,565504
2,689,617,356,771,474721,425113,245284,531219,380689,219652,475707,126736,274476,594441
3,182,996,338,290,33124,181272,61516,52780,992016,336648,288840,114244,98020,84100
4,179,236,191,536,32041,42244,34189,95944,55696,45076,126496,36481,102376,287296


In [98]:
from sklearn.preprocessing import StandardScaler

In [99]:
norm_scal = StandardScaler()

In [100]:
df3 = pd.DataFrame(index=df2.index, columns=df2.columns)

In [101]:
for col in df2.columns:
    df3[col] = norm_scal.fit_transform(df2[col].values.reshape(-1,1))

In [102]:
df3.head()

,A,B,C,D,AxA,AxB,AxC,AxD,BxB,BxC,BxD,CxC,CxD,DxD
0,1.524643,-1.472584,-1.489408,1.247081,1.849727,-0.811742,-0.837138,2.549205,-1.101843,-1.108144,-0.842511,-1.097849,-0.864636,1.366894
1,0.790658,1.397399,0.806733,0.874693,0.661849,1.851068,1.268851,1.361811,1.621344,1.878242,1.955959,0.681696,1.362519,0.784609
2,0.665429,0.404878,-0.493492,0.940818,0.489316,0.803529,-0.018121,1.297671,0.159171,-0.131467,1.028797,-0.688768,0.113325,0.882357
3,-1.098221,1.720142,-0.555736,-0.733190,-1.007422,-0.309169,-0.846751,-0.897239,2.215358,0.401287,0.176784,-0.730617,-0.685360,-0.841563
4,-1.108657,-0.917327,-1.064069,0.122955,-1.011093,-0.943582,-0.969972,-0.699218,-0.933938,-0.926419,-0.563418,-0.991127,-0.665644,-0.155171


In [103]:
df3.shape[1]

14

In [104]:
from sklearn.decomposition import PCA

In [105]:
pca = PCA(n_components=df3.shape[1])

In [106]:
pca.fit_transform(df3)

array([[-0.43589026, -0.15788098,  1.52213376, ..., -0.07963859,
         0.00849475,  0.10500093],
       [ 4.75053149, -0.6827325 , -0.89666753, ..., -0.15043615,
        -0.14832939,  0.17828828],
       [ 1.47734739, -1.16168224, -0.18221086, ...,  0.04287002,
        -0.01553272, -0.14151248],
       ...,
       [-1.63244436,  0.95491332,  0.31121076, ..., -0.07452253,
        -0.12813233, -0.09893741],
       [-0.4920575 ,  0.21700632,  1.85049877, ..., -0.14109343,
        -0.13280799, -0.10069964],
       [-1.96807145,  0.06405901,  0.26659957, ..., -0.03700894,
        -0.03455973, -0.12404066]])

In [107]:
vec_var = pca.explained_variance_

In [108]:
pca_comp = (vec_var>1).sum()

In [109]:
pca2 = PCA(n_components=pca_comp)

In [110]:
mat4 = pca2.fit_transform(df3)

In [111]:
mat4.shape

(10000, 4)

In [112]:
Z = df['A'] + df['C']*df['C']*df['C']

In [113]:
Z.head()

0       315368
1    392223893
2     45118705
3     38614654
4      6968050
dtype: int32

In [114]:
columns = ['col'+str(i) for i in range(mat4.shape[1]+1)]

In [115]:
df5 = pd.DataFrame(index=df1.index, columns=columns)

In [116]:
df5.loc[:,df5.columns[:-1]] = mat4
df5.loc[:,df5.columns[-1]]  = Z

In [117]:
df5.head()

,col0,col1,col2,col3,col4
0,-0.43589,-0.157881,1.52213,4.48228,315368
1,4.75053,-0.682732,-0.896668,-0.343822,392223893
2,1.47735,-1.16168,-0.182211,1.71394,45118705
3,-0.921408,-1.52455,-2.66489,-1.66502,38614654
4,-2.88949,-0.941152,0.633769,0.284991,6968050


# NN con dati originali

In [118]:
import torch

In [119]:
df.head()

,A,B,C,D
0,936,76,68,859
1,725,903,732,752
2,689,617,356,771
3,182,996,338,290
4,179,236,191,536


In [122]:
zscore = StandardScaler()
df_ = pd.DataFrame(zscore.fit_transform(df))

In [123]:
df_.head()

,0,1,2,3
0,1.524643,-1.472584,-1.489408,1.247081
1,0.790658,1.397399,0.806733,0.874693
2,0.665429,0.404878,-0.493492,0.940818
3,-1.098221,1.720142,-0.555736,-0.733190
4,-1.108657,-0.917327,-1.064069,0.122955


In [124]:
df_['Z'] = Z

In [125]:
df_.head()

,0,1,2,3,Z
0,1.524643,-1.472584,-1.489408,1.247081,315368
1,0.790658,1.397399,0.806733,0.874693,392223893
2,0.665429,0.404878,-0.493492,0.940818,45118705
3,-1.098221,1.720142,-0.555736,-0.733190,38614654
4,-1.108657,-0.917327,-1.064069,0.122955,6968050


In [126]:
in_data = df_.astype(np.float32)

In [128]:
in_data_tens = torch.from_numpy(in_data.values)

In [129]:
from sklearn.model_selection import train_test_split

In [130]:
in_data_tens_train, in_data_tens_cv = train_test_split(in_data_tens, test_size=0.20)

In [131]:
x_train = in_data_tens_train[:,:-1]
y_train = in_data_tens_train[:,-1]
x_cv = in_data_tens_cv[:,:-1]
y_cv = in_data_tens_cv[:,-1]

x_train.shape,y_train.shape,x_cv.shape,y_cv.shape

(torch.Size([8000, 4]),
 torch.Size([8000]),
 torch.Size([2000, 4]),
 torch.Size([2000]))

In [132]:
y_train

tensor([1.3825e+07, 1.9610e+03, 9.4119e+08,  ..., 4.7438e+07, 2.5155e+07,
        1.8617e+08])

In [133]:
import torch
from torch import nn

In [134]:
nn_model = nn.Sequential(
    nn.Linear(x_train.shape[1],250),
    nn.ReLU(),
    nn.Linear(250,1),
    nn.ReLU(),
)

In [135]:
nn_model(x_cv)

tensor([[0.0000],
        [0.0160],
        [0.3816],
        ...,
        [0.1398],
        [0.0000],
        [0.0370]], grad_fn=<ReluBackward0>)

In [136]:
error = nn.MSELoss()

In [139]:
learning_rate = 100
optimizer = torch.optim.SGD(nn_model.parameters(), lr = learning_rate)

In [141]:
for i in range(1000):
    y_pred = nn_model(x_train) #calcolo l'uscita
    loss = error(y_pred, y_train.reshape(-1,1))  #calcolo l'errore
    loss.backward()        #calcolo del gradiente
    optimizer.step()       #aggiornamento dei parametri
    optimizer.zero_grad()  #azzeramento del gradiente
    if np.mod(i,100)==0:
        print(loss.data)

tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)
tensor(1.4312e+17)


In [142]:
y_pred.shape, y_train.reshape(-1,1).shape

(torch.Size([8000, 1]), torch.Size([8000, 1]))

In [60]:
error(y_pred, y_train)

tensor(1.0138, grad_fn=<MseLossBackward>)

In [90]:
for param in nn_model.parameters():
    print(param)

Parameter containing:
tensor([[ 3.3220e-01,  3.3267e-01, -3.3742e-01, -2.4557e-01],
        [ 3.4413e+01, -3.8392e+01,  3.1942e+01, -3.6865e+01],
        [ 5.9346e+02, -4.0683e+02,  1.7481e+01,  4.2063e+02],
        [ 4.0019e+01,  4.7625e+01, -1.6541e+01, -4.3849e+01],
        [ 2.6263e+03,  2.1071e+03,  2.8428e+03,  3.0673e+03],
        [-3.2013e+02, -3.0274e+01, -3.3539e+02, -1.3158e+02],
        [-4.7767e+03, -2.6226e+03, -8.6554e+03,  3.3441e+03],
        [-5.1630e+03, -3.5368e+03, -9.2527e+03,  3.1289e+03],
        [ 2.5509e+02,  2.5543e+02,  1.7682e+00, -1.0291e+02],
        [ 4.0302e+02,  4.1188e+02,  2.7658e+02, -2.4103e+02],
        [-1.1508e-01, -9.8981e-02, -8.0121e-02,  8.9604e-02],
        [ 1.1449e+00,  4.2494e+01, -1.8912e+02, -5.9564e+01],
        [ 1.0819e+03,  1.1021e+03,  1.4470e+03, -2.0250e+03],
        [ 2.3356e+02, -2.5787e+02,  7.0079e+02,  4.2047e+01],
        [ 1.4598e+02,  1.2092e+02,  8.7692e+01, -1.1459e+02],
        [-5.8454e+03, -3.0572e+03, -9.9112e+03, 